In [83]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

#### Implement the **FXBarrierOption** Class

In [73]:
class FXBarrierOption:
    """A class to model Foreign Exchange (FX) Barrier Options using the Black-Scholes framework.
       Supports vanilla options and barrier options (up-in, up-out, down-in, down-out)"""

    def __init__(self, spot_price, strike_price, time_to_maturity, volatility,
                 domestic_rate, foreign_rate, barrier_level=None, barrier_type=None):
        """
        Initializes the FXBarrierOption object with key financial parameters.

        :param spot_price: Current price of the underlying asset.
        :param strike_price: Option's strike price.
        :param time_to_maturity: Time to maturity (in years).
        :param volatility: Annualized volatility of the underlying asset.
        :param domestic_rate: Domestic risk-free interest rate (annualized).
        :param foreign_rate: Foreign risk-free interest rate (annualized).
        :param barrier_level: The price level of the barrier (if applicable).
        :param barrier_type: Type of barrier ('up-in', 'up-out', 'down-in', 'down-out').
        """
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.time_to_maturity = time_to_maturity
        self.volatility = volatility
        self.domestic_rate = domestic_rate
        self.foreign_rate = foreign_rate
        self.barrier_level = barrier_level
        self.barrier_type = barrier_type

    def compute_d1(self, spot_price=None):
        """
        Computes the Black-Scholes d1 term used in option pricing.

        :param spot_price: (Optional) Current spot price of the underlying asset.
        :return: Computed d1 value.
        """
        if spot_price is None:
            spot_price = self.spot_price

        d1 = (np.log(spot_price / self.strike_price) +
              (self.domestic_rate - self.foreign_rate + 0.5 * self.volatility ** 2) * self.time_to_maturity) / \
             (self.volatility * np.sqrt(self.time_to_maturity))
        return d1

    def compute_d2(self, d1=None):
        """ Computes the Black-Scholes d2 term
        :param d1: (Optional) Computed d1 value.
        :return: Computed d2 value. """
        if d1 is None:
            d1 = self.compute_d1()

        return d1 - self.volatility * np.sqrt(self.time_to_maturity)

    def calculate_vanilla_price(self, option_type='call'):
        """ Computes the price of a vanilla European option.
        :param option_type: 'call' for a call option, 'put' for a put option.
        :return: Vanilla option price"""
        d1 = self.compute_d1()
        d2 = self.compute_d2(d1)

        if option_type == 'call':
            price = (self.spot_price * np.exp(-self.foreign_rate * self.time_to_maturity) * norm.cdf(d1)) - \
                    (self.strike_price * np.exp(-self.domestic_rate * self.time_to_maturity) * norm.cdf(d2))
        elif option_type == 'put':
            price = (self.strike_price * np.exp(-self.domestic_rate * self.time_to_maturity) * norm.cdf(-d2)) - \
                    (self.spot_price * np.exp(-self.foreign_rate * self.time_to_maturity) * norm.cdf(-d1))
        else:
            raise ValueError("Invalid option type. Choose 'call' or 'put'.")

        return price


### Class Test: Price Vanilla option

In [80]:
# Create an FXBarrierOption instance
option = FXBarrierOption(spot_price=100,
                         strike_price=90,
                         time_to_maturity=3,
                         volatility=0.16,
                         domestic_rate=0.05,
                         foreign_rate=0.03)

# Compute vanilla option prices
call_price = option.calculate_vanilla_price(option_type='call')
put_price = option.calculate_vanilla_price(option_type='put')

# Print results
print(f"Vanilla Call Price: {call_price:.4f}")
print(f"Vanilla Put Price: {put_price:.4f}")


Vanilla Call Price: 17.8633
Vanilla Put Price: 3.9339


### **Explanation:**
- The call option has a higher value because **the spot price ($100$) is greater than the strike price ($90$).**
- The put option is cheaper since it is **out-of-the-money (OTM)**.
- A **longer time to maturity** (3 years) generally **increases option prices**.
- **Higher domestic interest rates favor call options**.

---